# Introduction
The purpose of this notebook is to provide an example of data cleaning and preparation. In this example, data will be pulled from the Social Security's data set of popular baby names found [here](https://www.ssa.gov/oact/babynames/limits.html). This data will be directly downloaded from the website and organized into one data frame that will be used for a streamlit app.

# Downloading the Data
First, we'll pull the data directly from the website found [here](https://www.ssa.gov/oact/babynames/limits.html). At first, I thought this was going to be a simple download using the code below...

In [1]:
import requests

# Basic download
url = "https://www.ssa.gov/oact/babynames/names.zip"
response = requests.get(url)

# Save to file
with open("downloaded_file.zip", "wb") as f:
    f.write(response.content)

# Check for errors
if response.status_code == 200:
    print("Download successful")
else:
    print(f"Error: {response.status_code}")

Error: 403


I was wrong. After, many code re-writes it looks like this website detects automated requests and blocks them. I'll try to get around this by using Selenium to simulate being an actual user. NOTE: This should be done with caution. This is a sure way to get kicked off a website if you abuse this power. This happened to me during one of my projects at Booz Allen. I used Selenium to webscrape [matweb](https://matweb.com/) and aggregate material information that I needed for the project. I learned the hard way that your IP address will be black listed if you extract too much data. For this project though, I just want to download one link and that's it. So let's begin. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import os

def download_with_selenium_custom_folder(redownload=False):
    if redownload==False:
        if os.path.exists(os.path.abspath("data/names.zip")):
            print("File already exists. Skipping download.")
            return
            
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
    
    # Set download directory
    download_dir = os.path.abspath("data")  # Downloads to your project's data folder
    os.makedirs(download_dir, exist_ok=True)
    
    prefs = {
        "download.default_directory": download_dir,
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        # Visit the main page first
        driver.get("https://www.ssa.gov/oact/babynames/limits.html")
        time.sleep(2)
        
        # Find and click the download link
        links = driver.find_elements(By.TAG_NAME, "a")
        for link in links:
            href = link.get_attribute("href")
            if href and "names.zip" in href:
                print(f"Downloading from: {href}")
                driver.get(href)
                time.sleep(5)  # Wait for download to complete
                print(f"File downloaded to: {download_dir}")
                break
                
    finally:
        driver.quit()

# Run the function
download_with_selenium_custom_folder()

Worked like a charm! Now it's time to restructre the data.

# Restructuring the Data
The data is in a zip folder so let's unzip it.

In [4]:
# unzip the file
import zipfile
zip_file_path = os.path.abspath("data/names.zip")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall("data")

Looks like the data was all stored in text files and there is a ReadMe that states:

> **National Data on the relative frequency of given names in the population of
> U.S. births where the individual has a Social Security Number**
> (Tabulated based on Social Security records as of March 2, 2025)
> 
> For each year of birth YYYY after 1879, we created a comma-delimited file called yobYYYY.txt. Each
> record in the individual annual files has the format "name,sex,number," where name is 2 to 15 characters,
> sex is M (male) or F (female) and "number" is the number of occurrences of the name. Each file is sorted
> first on sex and then on number of occurrences in descending order. When there is a tie on the number of
> occurrences, names are listed in alphabetical order. This sorting makes it easy to determine a name's rank.
> The first record for each sex has rank 1, the second record for each sex has rank 2, and so forth.
>
> To safeguard privacy, we restrict our list of names to those with at least 5 occurrences

Alright, we'll have to read in all of the txt files and aggregate them into one pandas data frame.

In [5]:
# read in all of the txt files
import pandas as pd

# get all of the txt files in the data folder
txt_files = [f for f in os.listdir("data") if f.endswith(".txt")]

# read in all of the txt files
df = pd.DataFrame()
for file in txt_files:
    df_add = pd.read_csv(os.path.join("data", file), header=None)
    df_add.columns = ["name", "sex", "total_count"]
    df_add["year"] = file.split("yob")[1].split(".txt")[0]
    df = pd.concat([df, df_add])

# convert year to int
df["year"] = df["year"].astype(int)

# drop duplicates and reset index
df = df.drop_duplicates(subset=["name", "sex", "year"])
df = df.reset_index(drop=True)

# sort by year, sex, and total_count
df = df.sort_values(by=["year", "sex", "total_count"], ascending=False)

df.head()

,name,sex,total_count,year
2135234,Liam,M,22164,2024
2135235,Noah,M,20337,2024
2135236,Oliver,M,15343,2024
2135237,Theodore,M,12011,2024
2135238,James,M,11793,2024


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2149477 entries, 2135234 to 941
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   name         object
 1   sex          object
 2   total_count  int64 
 3   year         int64 
dtypes: int64(2), object(2)
memory usage: 82.0+ MB


Let's also update the data so names are persistent each year even if it's total count is zero for a year. For example, if a unique name like Zyler was use 2 times in 2023 but not once in 2024. There isn't a row stating the total count for Zyler was 0 for 2024. I want to include these rows to help perform trend analysis down the road. The reasoning for this might not be apparent now, but you will see why this is important when we start developing projections for popularities of names.

In [7]:
names_sex = df[['name', 'sex']].drop_duplicates()
years = pd.DataFrame({'year': df['year'].drop_duplicates()})

# Cross join (cartesian product)
all_names = names_sex.assign(key=1).merge(years.assign(key=1), on='key').drop('key', axis=1)
all_names['total_count'] = 0

# merge the data frames
df = pd.concat([df, all_names])

# drop duplicates and reset index
df = df.drop_duplicates(subset=["name", "sex", "year"])
df = df.reset_index(drop=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16899750 entries, 0 to 16899749
Data columns (total 4 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   name         object
 1   sex          object
 2   total_count  int64 
 3   year         int64 
dtypes: int64(2), object(2)
memory usage: 515.7+ MB


Let's add a new feature that calculates the relative popularity of a name for each year.

In [8]:
# calculate the relative popularity of a name for each year
df["popularity_percent"] = df.groupby(["sex", "year"])["total_count"].transform("sum")
df["popularity_percent"] = df["total_count"] / df["popularity_percent"]

df['popularity_rank'] = df.groupby(['sex', 'year'])['popularity_percent'].rank(method='min', ascending=False).astype(int)
df.head()


,name,sex,total_count,year,popularity_percent,popularity_rank
0,Liam,M,22164,2024,0.012921,1
1,Noah,M,20337,2024,0.011856,2
2,Oliver,M,15343,2024,0.008945,3
3,Theodore,M,12011,2024,0.007002,4
4,James,M,11793,2024,0.006875,5


# Create SQLite Database
And now we have one giant data frame. Let's save it to a SQLite database and use it for the Streamlit app.

In [5]:
import sqlite3

db_path = "data/names.db"
if os.path.exists(db_path)==False:
    # create a connection to the database
    conn = sqlite3.connect("data/names.db")

    # save the data frame to a SQLite database  
    df.to_sql("names", conn, if_exists="replace", index=False)

    # Create indexes for better performance
    cursor = conn.cursor()
    cursor.execute('CREATE INDEX idx_name ON names(name)')
    cursor.execute('CREATE INDEX idx_sex ON names(sex)')
    cursor.execute('CREATE INDEX idx_year ON names(year)')
    conn.commit()

    # close the connection
    conn.close()